In [1]:
from keras.datasets import imdb
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import gensim
import os, re, csv, math, codecs
num_words=30000
(training_data, training_targets), (testing_data, testing_targets) = imdb.load_data(num_words=num_words+2,)
data = np.concatenate((training_data, testing_data), axis=0)
targets = np.concatenate((training_targets, testing_targets), axis=0)

17473536/17464789 [==============================] - 0s 0us/step


In [2]:
num_words=len(np.unique(np.hstack(data)))
print("Categories:", np.unique(targets))
print("Number of unique words:", num_words)

Categories: [0 1]
Number of unique words: 30000


Agregar este archivo a la carpeta de google drive

https://drive.google.com/file/d/12F_EmUdgY_eo86__UzIgrtyzlkK1NA1B/view?usp=sharing

El archivo sale de: https://fasttext.cc/docs/en/english-vectors.html

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
length = [len(i) for i in data]
print("Average Review length:", np.mean(length))
print("Standard Deviation:", round(np.std(length)))

Average Review length: 234.75892
Standard Deviation: 173


In [5]:
print("Label:", targets[0])

Label: 1
print(data[0])

Label: 1
[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 22665, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 21631, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


In [6]:
index = imdb.get_word_index()
reverse_index = dict([(value, key) for (key, value) in index.items()]) 
decoded = " ".join( [reverse_index.get(i - 3, "#") for i in data[1]] )
print(decoded)

1654784/1641221 [==============================] - 0s 0us/step
# big hair big boobs bad music and a giant safety pin these are the words to best describe this terrible movie i love cheesy horror movies and i've seen hundreds but this had got to be on of the worst ever made the plot is paper thin and ridiculous the acting is an abomination the script is completely laughable the best is the end showdown with the cop and how he worked out who the killer is it's just so damn terribly written the clothes are sickening and funny in equal measures the hair is big lots of boobs bounce men wear those cut tee shirts that show off their stomachs sickening that men actually wore them and the music is just # trash that plays over and over again in almost every scene there is trashy music boobs and paramedics taking away bodies and the gym still doesn't close for # all joking aside this is a truly bad film whose only charm is to look back on the disaster that was the 80's and have a good old laugh a

In [7]:
from tqdm import tqdm

In [8]:
#load embeddings

print('loading word embeddings...')
embeddings_index = {}
f = codecs.open('/content/drive/MyDrive/wiki-news-300d-1M.vec', encoding='utf-8')
for line in tqdm(f):
    values = line.rstrip().rsplit(' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('found %s word vectors' % len(embeddings_index))

loading word embeddings...


999995it [01:19, 12538.07it/s]

found 999995 word vectors


In [10]:
embeddings_index["home"]

array([ 1.576e-01,  2.110e-02,  1.950e-02,  9.590e-02,  6.000e-04,
        8.230e-02, -3.850e-02, -4.880e-02, -2.200e-03, -5.000e-02,
       -1.260e-02, -8.210e-02, -2.240e-02,  8.290e-02, -4.530e-02,
       -7.700e-03, -1.390e-02, -6.670e-02,  5.500e-02, -6.070e-02,
       -1.870e-02, -5.920e-02, -1.501e-01,  1.910e-01, -1.181e-01,
       -3.200e-02, -1.066e-01,  2.630e-02, -3.070e-02, -4.810e-02,
       -5.270e-02,  4.270e-02, -6.970e-02, -3.890e-02,  1.720e-02,
       -1.270e-02,  3.220e-02,  5.730e-02,  2.380e-02,  1.720e-02,
       -4.620e-02,  6.500e-03, -2.212e-01, -4.700e-02,  4.970e-02,
       -7.460e-02,  8.080e-02, -3.000e-02, -1.008e-01, -7.080e-02,
        1.840e-02,  1.363e-01, -6.166e-01,  2.990e-02, -8.810e-02,
       -6.800e-02, -7.730e-02,  6.740e-02,  9.540e-02, -1.740e-02,
        3.680e-02, -9.040e-02, -6.720e-02, -2.400e-02,  5.550e-02,
       -2.070e-02,  4.300e-02, -7.760e-02,  1.064e-01,  6.110e-02,
        7.950e-02, -7.790e-02,  7.080e-02,  3.990e-02, -9.060e

In [11]:
embed_dim=300
embedding_matrix=np.zeros([num_words+4,embed_dim])
for word, idx in index.items():
  if idx <= num_words and word in embeddings_index:
    embedding_matrix[idx+3,:]=embeddings_index[word]

In [12]:
maxlen=1000

In [13]:
data = pad_sequences(data,maxlen=maxlen)

In [14]:
len(data[0])

1000

In [15]:
len(data[1])

1000

In [16]:
data=np.array(data)

In [17]:
data.shape

(50000, 1000)

In [18]:
import tensorflow as tf
from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dropout, Dense
from keras.models import Sequential
from keras import optimizers
nb_words=num_words+4
num_filters=64
model = Sequential()
model.add(Embedding(nb_words, embed_dim, weights=[embedding_matrix], input_length=maxlen, trainable=False))
model.add(Conv1D(num_filters, 7, activation='relu', padding='same'))
model.add(MaxPooling1D(2))
model.add(Conv1D(num_filters*2, 7, activation='relu', padding='same'))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  #multi-label (k-hot encoding)

adam = tf.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1000, 300)         9001200   
                                                                 
 conv1d (Conv1D)             (None, 1000, 64)          134464    
                                                                 
 max_pooling1d (MaxPooling1D  (None, 500, 64)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 500, 128)          57472     
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                                  
                                                                 
 dropout (Dropout)           (None, 128)               0

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [19]:
model.fit(data,targets,batch_size=32,epochs=20,validation_split=0.2)

Epoch 1/20
1250/1250 [==============================] - 30s 14ms/step - loss: 0.3959 - accuracy: 0.8124 - val_loss: 0.2725 - val_accuracy: 0.8833
Epoch 2/20
1250/1250 [==============================] - 16s 13ms/step - loss: 0.2749 - accuracy: 0.8875 - val_loss: 0.2508 - val_accuracy: 0.8982
Epoch 3/20
1250/1250 [==============================] - 16s 13ms/step - loss: 0.2345 - accuracy: 0.9065 - val_loss: 0.2450 - val_accuracy: 0.9018
Epoch 4/20
1250/1250 [==============================] - 16s 13ms/step - loss: 0.1990 - accuracy: 0.9217 - val_loss: 0.2608 - val_accuracy: 0.8919
Epoch 5/20
1250/1250 [==============================] - 16s 13ms/step - loss: 0.1642 - accuracy: 0.9377 - val_loss: 0.2604 - val_accuracy: 0.8972
Epoch 6/20
1250/1250 [==============================] - 18s 14ms/step - loss: 0.1373 - accuracy: 0.9468 - val_loss: 0.2647 - val_accuracy: 0.8991
Epoch 7/20
1250/1250 [==============================] - 18s 14ms/step - loss: 0.1115 - accuracy: 0.9574 - val_loss: 0.2900 -

In [20]:
embedding_matrix

array([[ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  0.    ],
       [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  0.    ],
       [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  0.    ],
       ...,
       [-0.1016, -0.0326, -0.0645, ...,  0.1472,  0.0353,  0.0965],
       [-0.0605, -0.0081,  0.1003, ...,  0.126 , -0.0304, -0.0316],
       [-0.2191, -0.0352, -0.0829, ..., -0.1367, -0.2115, -0.0451]])